# Error Analisys using RNN's on original Dataset

### First we're going to tune up a simple rnn to analize the generalization error

### Setting up variables, library and data

In [0]:
import pandas as pd
import numpy as np

# Run this cell to mount your Google Drive.
# from google.colab import drive
# drive.mount('/content/drive')



In [0]:
def load_data(label):
  #label = 'all'
  print("Loading data with all components")
  names_X = ['X_train_labels_niveladas.csv', 'X_test.csv']
  names_y = ['y_train_labels_niveladas.csv', 'y_test.csv']

  X = []
  y = []

  for dataset in names_X:
    dataframe = pd.read_csv(''.join([path,dataset]), index_col=0)
    dataframe = dataframe.iloc[:, dataframe.columns.str.contains(features)]
    #X.append(np.array(dataframe))
    X.append(dataframe)

  for dataset in names_y:
    dataframe = pd.read_csv(''.join([path,dataset]),index_col=0)
    #y.append(np.array(dataframe))
    y.append(dataframe)

  print('Shape X_train: ', np.shape(X[0]))
  print('Shape X_test : ', np.shape(X[1]))
  print('Shape y_train: ', np.shape(y[0]))
  print('Shape y_test : ', np.shape(y[1]))

  #return X_train, X_test, y_train, y_test
  return X[0], X[1], y[0], y[1], features # X_train, X_test, y_train, y_test

In [0]:
def reshape_for_lstm(X_reshape):
  print("Reshaping for LSTM")
  X_reshape = np.array(X_reshape)
  #X_reshape = np.array(X_train)[:,:-1]
  number_of_features = parameters.count('|') + 1

  shape_input = int(np.shape(X_reshape)[1]/number_of_features)
  X_new = np.array([])

  for example in X_reshape:
    # split data for each component, i.e., [fx, fy, fz]
    X = np.split(example, number_of_features)
    # reshapes each component for LSTM shape
    for i, x in enumerate(X):
      X[i] = np.reshape(x, (shape_input, 1))

    # concatenate all components with new shape and transpose it to be in (n_experiment, timesteps, components)
    X_example = np.concatenate([[x for x in X]])
    X_example = np.transpose(X_example)
    if X_new.shape == (0,):
      X_new = X_example
    else:
      X_new = np.concatenate((X_new, X_example))

  print("X_new.shape = ", X_new.shape)
  #print("new y data shape = ", X_test_full.shape)

  return X_new

In [0]:
## Data path handlers
#TRAIN_TEST_SET_PATH = '../../Data/train_test/'
path = r'/content/drive/My Drive/USP/Doutorado/Artigos - Escrita/2020 - IROS/Datasets/SMOTe_Nivelado/all_components/'
META_DATA_PATH = r'/content/drive/My Drive/USP/Doutorado/Artigos - Escrita/2020 - IROS/Datasets/SMOTe_Nivelado/all_components/glahr_meta/'
AUG_SET_PATH = '../../Data/aug_data_all/'
MODEL_PATH = r'/content/drive/My Drive/USP/Doutorado/Artigos - Escrita/2020 - IROS/Datasets/SMOTe_Nivelado/all_components/glahr_models/'
IMAGES_PATH = r'/content/drive/My Drive/USP/Doutorado/Artigos - Escrita/2020 - IROS/Datasets/SMOTe_Nivelado/all_components/glahr_images/'

## Selecting the desired features
parameters = 'fx|fy|fz|mx|my|mz'

In [0]:
### Getting data and selecting features
#X_train = pd.read_csv(TRAIN_TEST_SET_PATH+'X_train.csv',index_col=0)
#y_train = pd.read_csv(TRAIN_TEST_SET_PATH+'y_train.csv',index_col=0)
#X_test = pd.read_csv(TRAIN_TEST_SET_PATH+'X_test.csv',index_col=0)
#y_test = pd.read_csv(TRAIN_TEST_SET_PATH+'y_test.csv',index_col=0)

X_train, X_test, y_train, y_test, features = load_data(parameters)
X_train['labels'] = y_train.copy()

In [0]:
### Creating the validation set
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=27)
#for train, val in split.split(X_train, X_train['labels']):
for train, val in split.split(X_train, X_train['labels']):
    X_train_vl = X_train.iloc[train].copy()
    X_val = X_train.iloc[val].copy()
    
y_train_vl = X_train_vl['labels'].copy()
y_val = X_val['labels'].copy()

X_train_vl = X_train_vl.iloc[:, ~X_train_vl.columns.str.contains('labels')]
X_val = X_val.iloc[:, ~X_val.columns.str.contains('labels')]
#X_train = X_train.iloc[:, ~X_train.columns.str.contains('labels')]

y_train_vl = np.array(y_train_vl)-1
y_val = np.array(y_val)-1
y_test = np.array(y_test)-1
y_train = np.array(y_train)-1

print("X_train_vl.shape = ", X_train_vl.shape)
print("X_val.shape = ", X_val.shape)

print("y_train_vl.shape = ", y_train_vl.shape)
print("y_val.shape = ", y_val.shape)

In [0]:
### Standardizing the data
#from sklearn.preprocessing import StandardScaler

#std_scaler = StandardScaler()

#X_train = std_scaler.fit_transform(X_train)
#X_test = std_scaler.transform(X_test)
#X_train_vl = std_scaler.transform(X_train_vl)
#X_val = std_scaler.transform(X_val)

# AJUSTAR AQUI
X_train = X_train/30
X_test = X_test/30
X_train_vl = X_train_vl/30
X_val = X_val/30
print("X_train.shape = ", X_train.shape)
print("X_test.shape = ", X_test.shape)
print("X_train_vl.shape = ", X_train_vl.shape)
print("X_val.shape = ", X_val.shape)

In [0]:
### Reshaping data for LSTM
X_train = reshape_for_lstm(np.array(X_train)[:,:-1])
X_test = reshape_for_lstm(X_test)
X_train_vl = reshape_for_lstm(X_train_vl)
X_val = reshape_for_lstm(X_val)

print("\n\n\nX_train.shape = ", X_train.shape)
print("X_test.shape = ", X_test.shape)
print("X_train_vl.shape = ", X_train_vl.shape)
print("X_val.shape = ", X_val.shape)

### Setting up the mlp model and callbacks

In [0]:
### Training a simple rnn to use as reference
import tensorflow as tf
from tensorflow import keras

In [0]:
'''
### Model sketch
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[1556]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(3, activation='softmax'))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy'
                 ,optimizer=optimizer
                 ,metrics=['accuracy'])
    return model

mlp_simple = keras.wrappers.scikit_learn.KerasClassifier(build_model)
'''

In [0]:
### Model LSTM sketch
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=1556, features=6, output_shape=3, dropout_com=0.0, dropout_rec=0.0):

  model = keras.models.Sequential()
  # input layer
  if n_hidden == 0:
    model.add(keras.layers.LSTM(units=n_neurons, input_shape=(input_shape,features), return_sequences=False, dropout=dropout_com))
  else:
    model.add(keras.layers.LSTM(units=n_neurons, input_shape=(input_shape,features), return_sequences=True, dropout=dropout_com, recurrent_dropout=dropout_rec))
    if n_hidden > 1:
      for layer in range(n_hidden-1):
        model.add(keras.layers.LSTM(units=n_neurons, return_sequences=True, dropout=dropout_com, recurrent_dropout=dropout_rec))
      else:
        model.add(keras.layers.LSTM(units=n_neurons, return_sequences=False, dropout=dropout_com))

  # output layer
  model.add(keras.layers.Dense(output_shape, activation='softmax'))
  optimizer = keras.optimizers.SGD(lr=learning_rate)
  model.compile(loss='sparse_categorical_crossentropy'
                ,optimizer=optimizer
                ,metrics=['accuracy'])
  print(model.summary())
  return model

lstm_model = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [0]:
### Setting Callbacks
early_stopping = keras.callbacks.EarlyStopping(patience=15,
                                               restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint(MODEL_PATH+"lstm_simple.h5",
                                                save_best_only=True)                                     

get_stats = pd.DataFrame([])
get_stats.to_csv(META_DATA_PATH+'lstm_error_analysis.csv')

class GetLossAnalysis(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        get_stats = pd.read_csv(META_DATA_PATH+'lstm_error_analysis.csv', index_col=0)
        
        if len(get_stats.columns) == 0:
            get_stats.columns = ['epoch', 'train_loss', 'test_loss' ]
            
        get_stats['epoch'] = epoch
        get_stats['train_loss'] = logs.get('loss')
        get_stats['test_loss'] = logs.get('val_loss')
        
        get_stats.to_csv(META_DATA_PATH+'lstm_error_analysis.csv')

loss_analisys_cb = GetLossAnalysis()

### Hyperparameter optimization

In [0]:
### Setting parameters range for hyperparameter optimzation
### Here we will use random search as the optimization method
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV


params = {
    'n_hidden': np.arange(1,5),
    'n_neurons': np.arange(1,32),
    'learning_rate': reciprocal(3e-4, 3e-2),
    'dropout_com': np.arange(0,6)/10,
    'dropout_rec': np.arange(0,6)/10
    #'input_shape': [X_train_vl.shape[1]]
}

In [0]:
### Hyperparameter Optimization
rnd_search = RandomizedSearchCV(lstm_model, params, n_iter=10, cv=3, n_jobs=1)
print("X_train_vl.shape = ", X_train_vl.shape)
print("y_train_vl.shape = ", y_train_vl.shape)
rnd_search.fit(X_train_vl, y_train_vl, epochs=100,
               validation_split=0.15,
               callbacks=[early_stopping, checkpoint_cb])

In [0]:
### Best validation score
rnd_search.best_score_

In [0]:
### Best set of hyperparameters
best_parameters = pd.DataFrame(rnd_search.best_params_, index=['values'])
best_parameters.to_csv(MODEL_PATH+'mlp_simple_bp.csv')

rnd_search.best_params_

In [0]:
pd.read_csv(MODEL_PATH+'mlp_simple_bp.csv', usecols=['learning_rate']).iloc[0]

### Training the reference model

In [0]:
### Selecting and retraining the best model
best = rnd_search.best_estimator_.model
best.fit(X_train, y_train, epochs=200,
         validation_data=(X_test, y_test),
         callbacks=[early_stopping])

In [0]:
### Model evaluation in accuracy terms
best.evaluate(X_test, y_test)

## Error analysis

### Checking performance improvement feeding a stream of data

In [0]:
### Here we're going to retrain the model for each data set size.
### ranging from 1 to len(X_train)

## This variable will get the stats fo each training results
stats = pd.DataFrame([])

## The amount of data added for each training
data_batch = 1
number_of_steps = int(X_train.shape[0]/data_batch)

## Setting the prediction data frame which will store the predictions of each trained model
pred_columns = [(lambda x: 'size_'+str(x))(x) for x in np.arange(number_of_steps)*data_batch]
pred_shape = np.zeros(number_of_steps*y_test.shape[0]).reshape(y_test.shape[0], number_of_steps)
predictions = pd.DataFrame(pred_shape, columns=pred_columns)

## Recreating the optimized model but with re-initialized weights in order to fit them with each data size
config = keras.models.load_model(MODEL_PATH+'mlp_simple.h5').get_config()
mlp_model = keras.Sequential.from_config(config)

optimizer = keras.optimizers.SGD(lr=pd.read_csv(MODEL_PATH+'mlp_simple_bp.csv', 
                                                usecols=['learning_rate'])['learning_rate'].iloc[0])

mlp_model.compile(optimizer=optimizer,
                  metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

init_weights = mlp_model.get_weights()

## Looping through the data size and training the mdoel for each
for i in range(1, number_of_steps):
    mlp_model.set_weights(init_weights)
    
    if i*data_batch > X_train.shape[0]-1:
        history = mlp_model.fit(X_train[:X_train.shape[0]-1,:], y_train[:y_train.shape[0]-1], epochs=100,
                                validation_data=(X_test, y_test),
                                callbacks=[early_stopping], workers=8)
    
        predictions['size_'+str(i*data_batch)] = mlp_model.predict(X_test, y_test)
    
        temp = pd.DataFrame(history.history)
        stats = pd.append([stats, temp.iloc[temp.shape[0]-1,:]])
        break
    
    history = mlp_model.fit(X_train[:i*data_batch,:], y_train[:i*data_batch], epochs=100,
                            validation_data=(X_test, y_test),
                            callbacks=[early_stopping], workers=8)
    
    predictions['size_'+str(i)] = np.argmax(mlp_model.predict(X_test), axis=1)
    
    temp = pd.DataFrame(history.history)
    stats = stats.append(temp.iloc[temp.shape[0]-1,:])

In [0]:
### Structure of the stats data frame
stats.head(5)

In [0]:
### Storing training stats and predictions
stats.to_csv(META_DATA_PATH+'orig_error_analysis.csv')
predictions.to_csv(META_DATA_PATH+'orig_error_analysis_batch'+str(data_batch)+'.csv')

In [0]:
### Filtering only the accuracy results from the training and validation data sets
#stats_filter = stats.iloc[stats.index.str.contains('accuracy'),:]

## getting the indexes for accuracy and validation set accuracy
#indexes = np.arange(stats_filter.shape[0]-1)
#index_slct = indexes[indexes%2==0]

## Setting the accuracy arrays for validation and training set
#acc = stats_filter.iloc[index_slct]
#val_acc = stats_filter.iloc[index_slct+1]

## Plots Section

### Original Data Set

In [0]:
### Setting plotting libraries
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib notebook

In [0]:
### This cell is meant to aglomerate the data into mean values of specified sized steps
### The objective is to generate cleaner and more concise plots
train_acc = np.array([1])
test_acc = np.array([0])

acc = np.array(stats['accuracy'])
val_acc = np.array(stats['val_accuracy'])

mean_step = 20

for i in range(int(acc.shape[0]/mean_step)):
    if (i+1)*mean_step > acc.shape[0]-1:
            acc_mean = acc[i*mean_step:acc.shape[0]-1].mean()
            val_acc_mean = acc[i*mean_step:val_acc.shape[0]-1].mean()
            
            train_acc = np.concatenate([train_acc, acc_mean], axis=0)
            test_acc = np.concatenate([test_acc, val_acc_mean], axis=0)
            break
    
    acc_mean = acc[i*mean_step:(i+1)*mean_step].mean()
    val_acc_mean = val_acc[i*mean_step:(i+1)*mean_step].mean()
    
        
    train_acc = np.append(train_acc, values=[acc_mean], axis=0)
    test_acc = np.append(test_acc, values=[val_acc_mean], axis=0)


In [0]:
### Error analysis plot
fig = plt.figure(figsize=(8,6))
fig.suptitle('Orignal Dataset Error Analysis', fontweight='bold', fontsize=14)

ax = fig.add_subplot(1,1,1)

x_axis = np.arange(train_acc.shape[0])
x_axis = mean_step*x_axis

ax.plot(x_axis, 100*train_acc, color='b', label='Train Set Accuracy')
ax.plot(x_axis, 100*test_acc, color='r', label='Test Set Accuracy')

ax.set_xlim(0, x_axis[len(x_axis)-1])
ax.set_ylim(0,100)

ax.set_xlabel('Quantity of data')
ax.set_ylabel('Accuracy %')
ax.set_title('Model performance vs quantity of data', fontweight='bold', fontsize=12)

ax.legend(loc='lower right')

plt.grid()
plt.show()

#### From the graph above we can see we have a significant difference between test and training error. This means the difference in generalization error is probably due to overfitting.


In [0]:
y_test = np.array(y_test)
jammed_idx = np.where(y_test==2)



## Confusion mastrix, recall, precision and AUC ROC

In [0]:
fig.savefig(IMAGES_PATH+'error_analysis_orig.png', dpi=600, bbox_inches='tight', figsize=(8,6))
fig.savefig(IMAGES_PATH+'error_analysis_orig.jpg', dpi=600, bbox_inches='tight', figsize=(8,6))